# Embedding Upsampling
## Overview
Hello! We are excited to share this take-home assignment with you. Please read the problem
statement below and raise questions if any. We are happy to clarify over email or a quick call.
Please try to get back to us in a week. After your solution is ready, please share it with us and
schedule a 1-hour meeting to discuss your solution

## Prompt
At Connectly, ML engineers focus on both modeling and infra work. This task is designed to test
your ability to design, build, train, and ship ML models

## Task
Your task is to build a model and go through the full ML lifecycle. You’re given a problem to
solve with an ML model. You’ll need to take the model through the full ML lifecycle from ideation
to deploymen

## ML Problem
We have taken 1536 Dimensional OpenAI embeddings and applied a mystery affine
transformation to 32 dimensions. Your job is to build a model that given the 32-dimension
embeddings can upsample them back to 1536-dimensions in a way that preserves the cosine
angles between vectors of the original 1536-dimension embeddings. The upsampled vectors
should be of unit length. This model is approximating the inverse of the mystery transform
(mystery transform is not perfectly invertable)

We will provide:

1. Training Embeddings: This contains 12130 32-dimensional unit length embeddings
representing as a matrix of shape (12130,32). File:
https://cdn.connectly.ai/interview_prompts/ml_embedding_upsample/projected_train_embs.npy



1. Cosine Angle Similarity Matrix: This is a (12130, 12130) where the value at position ij
corresponds to the cosine angle <embedding_i, embedding_j>. Since these are unit
vectors they were calculated as dot<embedding_i, embedding_j>. These were
calculated on the original 1536-Dimensional embeddings NOT the 32-Dimensional
projected embeddings. File:
https://cdn.connectly.ai/interview_prompts/ml_embedding_upsample/og_train_cos_theta.npy


**We recommend starting with the simplest possible model and creating the service before
optimizing the mode**

In [1]:
import numpy as np
og = np.load('og_train_cos_theta.npy')
print(og.shape)

projected = np.load('projected_train_embs.npy')
print(projected.shape)

(12130, 12130)
(12130, 32)


In [2]:
# Split the data into training and test sets
projected_train = projected[: 10000]
projected_test = projected[10000: ]

og_train = og[:10000, :10000]
og_test = og[10000:, 10000: ]

print(projected_train.shape)
print(projected_test.shape)

print(og_train.shape)
print(og_test.shape)


(10000, 32)
(2130, 32)
(10000, 10000)
(2130, 2130)


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class FiveLayerNetwork(nn.Module):
    def __init__(self):
        super(FiveLayerNetwork, self).__init__()

        # Define the layers
        self.fc1 = nn.Linear(32, 64)  # First hidden layer
        self.fc2 = nn.Linear(64, 128) # Second hidden layer
        self.fc3 = nn.Linear(128, 256) # Third hidden layer
        self.fc4 = nn.Linear(256, 512) # Fourth hidden layer
        self.fc5 = nn.Linear(512, 1536) # Output layer

    def forward(self, x):
        # Pass the input tensor through each of the layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x


In [4]:
def rowwise_mse(matrix1, matrix2):
    if matrix1.shape != matrix2.shape:
        raise ValueError("Both matrices should have the same shape.")
    
    matrix2 = torch.from_numpy(matrix2).to(matrix1.device)  # Convert to the same device as matrix1
    mse_per_row = torch.mean((matrix1 - matrix2)**2, axis=1)
    return mse_per_row

def get_unit_vector(tensor):
    return tensor / torch.norm(tensor, dim=1, keepdim=True)

def custom_loss(upsampled_vectors, original_cosine_similarity_subset):
    upsampled_cosine_similarity_matrix = F.cosine_similarity(upsampled_vectors.unsqueeze(0), upsampled_vectors.unsqueeze(1), dim=2)
    cosine_similarity_loss = rowwise_mse(upsampled_cosine_similarity_matrix, original_cosine_similarity_subset)
    return cosine_similarity_loss


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
import torch.optim as optim


# Create DataLoader for training data
train_data = TensorDataset(torch.from_numpy(projected_train).float())
train_loader = DataLoader(train_data, batch_size=10, shuffle=False)
model = FiveLayerNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Initialize variable to store the previous epoch's loss
prev_epoch_loss = float('inf')  # Set to infinity initially so that any loss will be smaller

# Training loop
for epoch in range(10):
    # Load the best model's weights if it exists
    try:
        model.load_state_dict(torch.load('best_model.pth'))
    except FileNotFoundError:
        pass  # File will not exist for the first epoch
    
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_data = batch[0]
        upsampled_vectors = model(input_data)
        upsampled_vectors = get_unit_vector(upsampled_vectors)
        
        batch_indices = torch.arange(i * len(input_data), (i + 1) * len(input_data))
        
        # Limit batch_indices to the actual size of og_train
        batch_indices = torch.clamp(batch_indices, 0, len(og_train) - 1)

        # Retrieve the subset of `og_train` using the batch indices
        original_cosine_similarity_subset = og_train[batch_indices][:, batch_indices]
        
        row_loss = custom_loss(upsampled_vectors, original_cosine_similarity_subset)
        loss = row_loss.mean()
        
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")  # Fixed the typo from 'Item()' to 'item()'

    # Check if this loss is smaller than the previous epoch's loss
    if loss.item() < prev_epoch_loss:
        prev_epoch_loss = loss.item()
        print(f"Loss decreased to {prev_epoch_loss} from previous epoch. Saving model...")
        torch.save(model.state_dict(), 'best_model.pth')


Epoch 1, Loss: 0.001135932443457768
Loss decreased to 0.001135932443457768 from previous epoch. Saving model...
Epoch 2, Loss: 0.0009952715964652277
Loss decreased to 0.0009952715964652277 from previous epoch. Saving model...
Epoch 3, Loss: 0.0005349121509581653
Loss decreased to 0.0005349121509581653 from previous epoch. Saving model...
Epoch 4, Loss: 0.000837039769259176
Epoch 5, Loss: 0.0007622937694005988
Epoch 6, Loss: 0.0008035679792940608
Epoch 7, Loss: 0.0009542559251944317
Epoch 8, Loss: 0.0008885732570207119
Epoch 9, Loss: 0.0007306090493748895
Epoch 10, Loss: 0.0005963255495179799


In [6]:
test_data = torch.from_numpy(projected_test).float()
print(test_data.shape)

torch.Size([2130, 32])


In [8]:
from torch import no_grad

# Testing (evaluation)

# Forward pass on test data
model = FiveLayerNetwork()
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Initialize test loss to 0
total_test_loss = 0

# Number of samples in test_data
N = len(test_data)

# Size of each smaller square matrix
m = 100  # Set this to the size you want

# Calculate the number of blocks
num_blocks = N // m

with no_grad():
    upsampled_test_vectors = model(test_data)
    upsampled_test_vectors = get_unit_vector(upsampled_test_vectors)

    # Iterate over the blocks
    for i in range(num_blocks):
        for j in range(num_blocks):
            # Slice the matrices into smaller blocks
            slice_upsampled = upsampled_test_vectors[i*m:(i+1)*m, :]
            slice_og_test = og_test[i*m:(i+1)*m, j*m:(j+1)*m]

            # Calculate loss for the current block
            block_loss = custom_loss(slice_upsampled, slice_og_test)

            # Accumulate the loss
            total_test_loss += block_loss.mean().item()

# Average the accumulated loss
average_test_loss = total_test_loss / (num_blocks ** 2)

print(f"Average Test Loss: {average_test_loss}")


Average Test Loss: 0.0026575695940725098


# Service
You should package the model as a service such that you can pass in 32 dimensional
embeddings and it will return the upsampled embeddings (1536 dimensions). This service can
be run locally - no need to deploy to a cloud provider (unless you want to).
The service has 3 request types:
1. Batch request: takes in a `.txt` file with 1 embedding per row and each value in the
embedding is separated by a comma and writes the output upsampled embeddings to a
`.txt` file. You must assume the file is too large to be read into memory.
   * Request args:
     * input_filepath
     * output_filepath
   * Response args:
     * batch_request_id: unique identifier that can be used to check the status of
the batch request.
     * Example file 1 embedding per row with values separated by commas.
```
Q0.045, 0.34, 0.25, ..., -0.0094, 0.001
0.0035, -0.087, 0.1, ..., 0.011, 0.34
```
1. Batch status: returns the status of the batch job
    * Request args: batch_request_id
    * Response args:
      * status: Literal[‘COMPLETED’, ‘IN_PROGRESS’, ‘FAILED’]
      * num_records_processed: the number of records processed
2. Realtime request: optimize for low latency - the realtime request should be
prioritized over long running batch requests. If a long running batch request is in
progress then it should be paused to prioritize the realtime request.
   * Request args: input_embedding
   * Response args: upsampled_embedding